In [54]:
import requests
from bs4 import BeautifulSoup
import time

href_list = []  # リンク先を格納するリスト

# GoogleのGitHubページのURL
url = 'https://github.com/google'

# GoogleのGitHubページからリポジトリページへのリンクを取得
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

a_tags = soup.find_all('a')

# リポジトリページへのリンクを探す
for a in a_tags:
    href = a.get('href')  # href 属性を取得
    if href == '/orgs/google/repositories':
        href = "https://github.com" + href
        break

print(f"今回扱うリンク先: {href}")

# 1〜90ページまでURLをリストに格納
for i in range(1, 91):
    page_url = href + "?page=" + str(i)
    href_list.append(page_url)
    time.sleep(1)  # サーバーへの負荷を避けるため1秒待機

# リポジトリ番号をカウントする変数
repo_counter = 1

# リポジトリ名、主要言語、スター数を格納するリスト
repo_names = []
repo_languages = []
repo_stars = []

# 1ページから90ページまで繰り返し処理
for page_num in range(1, 91):
    # URLをページ番号でフォーマット
    url = href + "?page=" + str(page_num)
    
    # ページの内容を取得
    response = requests.get(url)

    # レスポンスが正常でない場合はスキップ
    if response.status_code != 200:
        print(f"{repo_counter}:欠損値")
        repo_counter += 1
        continue

    # レスポンスの内容を解析
    soup = BeautifulSoup(response.text, 'html.parser')

    li_tags = soup.find_all('li', class_='Box-sc-g0xbh4-0 ListItem-module__listItem--kHali')
    outer_div_tags = soup.find_all('div', class_='Box-sc-g0xbh4-0 ctIfAZ')

    # リポジトリ名を取得
    for div in li_tags:
        # div タグ内のさらに別の div タグ（class="list-view-item-title-container"）を検索
        inner_div_tags = div.find_all('div', class_='Box-sc-g0xbh4-0 jehEGV Title-module__container--l9xi7')

        for inner_div in inner_div_tags:
            a_tags = inner_div.find_all('a')  # <div> タグ内の <a> タグを検索

            for a in a_tags:
                span_tags = a.find_all('span')  # <a> タグ内の <span> タグを検索

                # <span> タグが存在する場合、そのテキストを出力
                for span in span_tags:
                    repo_name = span.get_text(strip=True)  # リポジトリ名を取得
                    repo_names.append(repo_name)  # リポジトリ名を格納

    # 主要言語の取得
    for outer_div in outer_div_tags:
        inner_div_tags = outer_div.find_all('div', class_='Box-sc-g0xbh4-0 fCvgBf')

        # 見つかった <div class="Box-sc-g0xbh4-0 fCvgBf"> タグ内の <span> タグを検索
        if inner_div_tags:
            for inner_div in inner_div_tags:
                span_tags = inner_div.find_all('span')

                # <span> タグが見つかった場合、その言語を取得
                if span_tags:
                    for span in span_tags:
                        language = span.get_text(strip=True)  # 主要言語を取得
                        repo_languages.append(language)  # 主要言語を格納

                    else:
                        repo_languages.append("記入なし")  # 言語なしとして格納                  
            else:
                repo_languages.append("記入なし")  # 言語なしとして格納
    else:
        repo_languages.append("記入なし")  # 言語なしとして格納

    # スター数の取得（aria-label 属性に基づく）
    a_tags = soup.find_all('a', attrs={'aria-label': True})

    for a in a_tags:
        aria_label = a.get('href')  # aria-label 属性を取得
        
        if '/stargazers' in aria_label.lower():  # aria-label に "stargazers" が含まれているか確認
            # <a> タグ内のテキストを取得（数字が含まれている場合）
            text = a.get_text(strip=True)
            
            if text.isdigit():  # テキストが数字だけの場合
                repo_stars.append(text)  # スター数を格納
            else:
                repo_stars.append("記入なし")  # スター数が記入されていない場合

    # サーバーへの負荷を避けるために少し待機
    time.sleep(0.5)

# リポジトリ名、主要言語、スター数を対応させて同時に出力
# リストの長さが一致していることを確認して、最初のリポジトリとその主要言語、スター数を表示
for i in range(min(len(repo_names), len(repo_languages), len(repo_stars))):
    print(f"{repo_counter}. {repo_names[i]} - {repo_languages[i]} - Stars: {repo_stars[i]}")
    repo_counter += 1

# 最終的な出力行数を表示
print(f"\n出力された行数: {repo_counter - 1}")


今回扱うリンク先: https://github.com/orgs/google/repositories
1. jscin - JavaScript - Stars: 39
2. clusterfuzz - 記入なし - Stars: 記入なし
3. fonts - 記入なし - Stars: 記入なし
4. syzkaller - Python - Stars: 記入なし
5. arolla - 記入なし - Stars: 14
6. fleetspeak - 記入なし - Stars: 138
7. zerocopy - HTML - Stars: 記入なし
8. docsy-example - 記入なし - Stars: 471
9. crubit - 記入なし - Stars: 720
10. hiba - Go - Stars: 371
11. site-kit-wp - 記入なし - Stars: 記入なし
12. tsunami-security-scanner-plugins - 記入なし - Stars: 878
13. workflow-graph - C++ - Stars: 110
14. dwh-migration-tools - 記入なし - Stars: 42
15. perfetto - 記入なし - Stars: 記入なし
16. benchmark - Go - Stars: 記入なし
17. oss-fuzz-gen - 記入なし - Stars: 922
18. android-fhir - 記入なし - Stars: 492
19. dawn - Rust - Stars: 413
20. tsl - 記入なし - Stars: 84
21. blockly - 記入なし - Stars: 記入なし
22. osv-scalibr - HTML - Stars: 92
23. wuffs - 記入なし - Stars: 記入なし
24. device-infra - 記入なし - Stars: 38
25. s2geometry - C++ - Stars: 記入なし
26. ground-platform - 記入なし - Stars: 204
27. koladata - 記入なし - Stars: 7
28. ang

In [34]:
import requests
from bs4 import BeautifulSoup
import time

href_list = []  # リンク先を格納するリスト

url = 'https://github.com/google'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

a_tags = soup.find_all('a')

# リポジトリページへのリンクを探す
for a in a_tags:
    href = a.get('href')  # href 属性を取得
    if href == '/orgs/google/repositories':
        href = "https://github.com" + href
        break

print(f"今回扱うリンク先: {href}")

# 1〜90ページまでURLをリストに格納
for i in range(1, 91):
    page_url = href + "?page=" + str(i)
    href_list.append(page_url)
    time.sleep(1)  # サーバーへの負荷を避けるため1秒待機

# リポジトリ番号をカウントする変数
repo_counter = 1

# リストのリンクを1つずつ処理
for href in href_list:
    # URLからページ内容を取得
    response = requests.get(href)
    
    # レスポンスの内容を解析
    soup = BeautifulSoup(response.text, 'html.parser')

    # class="Box-sc-g0xbh4-0 cCxlBe" の <div> タグを検索
    div_tags = soup.find_all('div', class_='Box-sc-g0xbh4-0 cCxlBe')

    # <div> タグ内の <a> タグ内の <span> タグを検索
    for div in div_tags:
        # div タグ内のさらに別の div タグ（class="list-view-item-title-container"）を検索
        inner_div_tags = div.find_all('div', class_='Box-sc-g0xbh4-0 jehEGV Title-module__container--l9xi7')

        # 各 inner div タグ内の <a> タグ内の <span> タグを検索
        for inner_div in inner_div_tags:
            a_tags = inner_div.find_all('a')  # <div> タグ内の <a> タグを検索

            # 各 <a> タグ内の <span> タグを検索
            for a in a_tags:
                span_tags = a.find_all('span')  # <a> タグ内の <span> タグを検索

                # <span> タグが存在する場合、そのテキストを出力
                for span in span_tags:
                    repo_name = span.get_text()  # リポジトリ名を取得

                    # 番号と一緒にリポジトリ名を出力
                    print(f"{repo_counter}. {repo_name}")

                    # カウントを増やす
                    repo_counter += 1

    # 各ページごとに適切な時間間隔をおく
    time.sleep(0.5)  # サーバーへの負荷を避けるため1秒待機

今回扱うリンク先: https://github.com/orgs/google/repositories
1. site-kit-wp
2. device-infra
3. s2geometry
4. ground-platform
5. koladata
6. angle
7. fonts
8. perfetto
9. osv-scalibr
10. fuzztest
11. ground-android
12. trillian
13. testrun
14. wasefire
15. dwh-migration-tools
16. tsl
17. oss-fuzz-gen
18. horologist
19. syzkaller
20. skia
21. shifter
22. riegeli
23. nomulus
24. xls
25. langfun
26. GoogleSignIn-iOS
27. seqio
28. distributed_point_functions
29. mug
30. orbit
31. nearby
32. osv.dev
33. XNNPACK
34. osv-scanner
35. mozc
36. rust-crate-audits
37. cel-cpp
38. aqt
39. saxml
40. crubit
41. dawn
42. boringssl
43. skia-buildbot
44. gvisor
45. oss-fuzz
46. j2cl
47. snippet-uiautomator
48. googleapis.dart
49. jsinterop-generator
50. automotive-design-compose
51. tensorstore
52. CFU-Playground
53. android-cuttlefish
54. jscin
55. orbax
56. gematria
57. crosvm
58. flink-connector-gcp
59. heir
60. filament
61. go-github
62. pigweed
63. copybara
64. earthengine-api
65. earthengine-catalog
66. l

In [41]:
import requests
from bs4 import BeautifulSoup
import time

# リポジトリページへのリンクを格納するリスト
href_list = []

# GoogleのGitHubページのURL
url = 'https://github.com/google'

# GoogleのGitHubページからリポジトリページへのリンクを取得
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# リポジトリページへのリンクを探す
for a in soup.find_all('a', href=True):
    if a['href'] == '/orgs/google/repositories':
        repo_page_url = 'https://github.com' + a['href']
        break

print(f"今回扱うリンク先: {repo_page_url}")

# 1〜90ページまでURLをリストに格納
for i in range(1, 91):
    page_url = f"{repo_page_url}?page={i}"
    href_list.append(page_url)
    time.sleep(1)  # サーバーへの負荷を避けるため1秒待機

# 出力された行数をカウントする変数
count = 1

# リストのリンクを1つずつ処理
for page_url in href_list:
    # ページの内容を取得
    response = requests.get(page_url)
    if response.status_code != 200:
        print(f"ページ {page_url} の読み込みに失敗しました。")
        continue

    # レスポンスの内容を解析
    soup = BeautifulSoup(response.text, 'html.parser')

    # <div class="Box-sc-g0xbh4-0 ctIfAZ"> を含む <div class="Box-sc-g0xbh4-0 fCvgBf"> を探す
    outer_div_tags = soup.find_all('div', class_='Box-sc-g0xbh4-0 ctIfAZ')

    # 見つかった外側の <div> タグを順番に処理
    if outer_div_tags:
        for outer_div in outer_div_tags:
            # この <div class="Box-sc-g0xbh4-0 ctIfAZ"> 内の <div class="Box-sc-g0xbh4-0 fCvgBf"> を探す
            inner_div_tags = outer_div.find_all('div', class_='Box-sc-g0xbh4-0 fCvgBf')

            # 見つかった <div class="Box-sc-g0xbh4-0 fCvgBf"> タグ内の <span> タグを検索
            if inner_div_tags:
                for inner_div in inner_div_tags:
                    span_tags = inner_div.find_all('span')
                    
                    # <span> タグが見つかった場合
                    if span_tags:
                        for span in span_tags:
                            print(f"{count} : {span.get_text(strip=True)}")  # <span> タグ内のテキスト（例: "Java"）を表示
                            count += 1  # カウントを増加
                    else:
                        print(f"{count}:記入なし")
                        count += 1  # カウントを増加
            else:
                print(f"{count}:記入なし")
                count += 1  # カウントを増加
    else:
        print(f"{count}:記入なし")
        count += 1  # カウントを増加




今回扱うリンク先: https://github.com/orgs/google/repositories
1 : Python
2 : HTML
3 : Kotlin
4 : C++
5 : C++
6 : C++
7 : TypeScript
8 : JavaScript
9 : Go
10 : C
11 : Java
12 : C++
13 : TypeScript
14 : C++
15 : C++
16 : C++
17 : Kotlin
18 : Go
19 : TypeScript
20 : Rust
21 : Java
22 : Kotlin
23 : Go
24 : C++
25 : Go
26 : C++
27 : Java
28 : C++
29 : Python
30 : Objective-C
31 : Python
32 : C++
33 : Java
34 : C++
35 : C++
36 : Python
37 : C
38 : Go
39 : C++
40:記入なし
41 : C++
42 : Python
43 : Python
44 : C++
45 : C++
46 : Go
47 : Go
48 : Shell
49 : Java
50 : Java
51 : Dart
52 : Java
53 : Rust
54 : C++
55 : Verilog
56 : C++
57 : JavaScript
58 : Python
59 : Python
60 : Rust
61 : Java
62 : C++
63 : C++
64 : Go
65 : C++
66 : Java
67 : JavaScript
68 : Jsonnet
69 : C++
70 : Java
71 : C
72 : Kotlin
73 : C++
74 : Python
75 : Python
76 : Objective-C
77 : Python
78 : Go
79 : Java
80 : C++
81 : Java
82 : LLVM
83 : TypeScript
84 : Python
85 : C++
86 : Python
87 : Rust
88 : Python
89 : Kotlin
90 : TypeScript
91 

In [ ]:
import requests
from bs4 import BeautifulSoup

# GoogleのGitHubページのURL
url = 'https://github.com/google'

# GoogleのGitHubページからリポジトリページへのリンクを取得
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# リポジトリページへのリンクを探す
for a in soup.find_all('a', href=True):
    if a['href'] == '/orgs/google/repositories':
        repo_page_url = 'https://github.com' + a['href']
        break
base_url = repo_page_url + "?page={}"


# 出力された行数をカウントする変数
line_count = 1

# 1ページから90ページまで繰り返し処理
for page_num in range(1, 91):
    # URLをページ番号でフォーマット
    url = base_url.format(page_num)

    # ページの内容を取得
    response = requests.get(url)

    # レスポンスが正常でない場合はスキップ
    if response.status_code != 200:
        print(f"{line_count}:欠損値")
        line_count += 1
        continue

    # レスポンスの内容を解析
    soup = BeautifulSoup(response.text, 'html.parser')

    # aria-label 属性を持ち、かつその中に "stars" が含まれている <a> タグを検索
    a_tags = soup.find_all('a', attrs={'aria-label': True})

    # 各ページのリポジトリとスター数を取得
    for a in a_tags:
        aria_label = a.get('href')  # aria-label 属性を取得
        
        if '/stargazers' in aria_label.lower():  # aria-label に "stars" が含まれているか確認
            # <a> タグ内のテキストを取得（数字が含まれている場合）
            text = a.get_text(strip=True)
            
            if text.isdigit():  # テキストが数字だけの場合
                print(f"{line_count}:{text}")
                line_count += 1  # 出力行数をカウント

            else:
                print(f"{line_count}:記入なし")
                line_count += 1




1:記入なし
2:記入なし
3:922
4:記入なし
5:492
6:413
7:記入なし
8:84
9:記入なし
10:記入なし
11:92
12:記入なし
13:38
14:記入なし
15:204
16:7
17:記入なし
18:731
19:245
20:記入なし
21:27
22:86
23:42
24:569
25:記入なし
26:43
27:419
28:記入なし
29:記入なし
30:476
31:513
32:558
33:73
34:369
35:記入なし
36:753
37:記入なし
38:記入なし
39:記入なし
40:記入なし
41:244
42:172
43:263
44:121
45:720
46:記入なし
47:138
48:記入なし
49:記入なし
50:記入なし
51:29
52:393
53:78
54:116
55:記入なし
56:469
57:380
58:39
59:296
60:76
61:813
62:7
63:324
64:記入なし
65:記入なし
66:339
67:記入なし
68:記入なし
69:91
70:376
71:記入なし
72:275
73:141
74:3
75:11
76:535
77:245
78:記入なし
79:記入なし
80:記入なし
81:14
82:記入なし
83:617
84:126
85:49
86:72
87:記入なし
88:記入なし
89:71
90:記入なし
91:記入なし
92:記入なし
93:記入なし
94:記入なし
95:393
96:記入なし
97:記入なし
98:601
99:584
100:記入なし
101:293
102:記入なし
103:記入なし
104:13
105:記入なし
106:640
107:記入なし
108:138
109:132
110:28
111:110
112:10
113:42
114:124
115:記入なし
116:記入なし
117:記入なし
118:153
119:記入なし
120:642
121:11
122:471
123:91
124:記入なし
125:記入なし
126:7
127:記入なし
128:14
129:177
130:13
131:115
132:記入なし
133:36
134:303
135:52
136:記入なし
1

In [52]:
import requests
from bs4 import BeautifulSoup
import time

# GoogleのGitHubページからリポジトリページへのリンクを取得
url = 'https://github.com/google'

# リポジトリページへのリンクを探す
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# リポジトリページのリンクを見つける
for a in soup.find_all('a', href=True):
    if a['href'] == '/orgs/google/repositories':
        repo_page_url = 'https://github.com' + a['href']
        break

# URLテンプレート
base_url = repo_page_url + "?page={}"

# 出力された行数をカウントする変数
line_count = 1

# 1ページから90ページまで繰り返し処理
for page_num in range(1, 91):
    # ページ番号をフォーマット
    url = base_url.format(page_num)

    # ページの内容を取得
    response = requests.get(url)

    # レスポンスが正常でない場合はスキップ
    if response.status_code != 200:
        print(f"{line_count}: 欠損値 (URL: {url})")
        line_count += 1
        continue

    # レスポンスの内容を解析
    soup = BeautifulSoup(response.text, 'html.parser')

    # リポジトリ名とスター数を取得する
    repo_data = soup.find_all('div', class_='Box-sc-g0xbh4-0 fCvgBf')

    for div in repo_data:
        # リポジトリ名を取得
        repo_name_tag = div.find('a')
        if repo_name_tag:
            repo_name = repo_name_tag.get_text(strip=True)
        else:
            repo_name = "記入なし"
        
        # 主要言語を取得（言語がない場合は"記入なし"）
        language_tag = div.find('span', class_='color-fg-muted')
        if language_tag:
            language = language_tag.get_text(strip=True)
        else:
            language = "記入なし"
        
        # スター数を取得
        star_tag = div.find('a', attrs={'aria-label': True})
        if star_tag and '/stargazers' in star_tag['aria-label']:
            star_count = star_tag.get_text(strip=True)
        else:
            star_count = "記入なし"

        # 1行で出力
        print(f"{line_count}, {repo_name}, {language}, {star_count}")
        
        # 行数をカウント
        line_count += 1

    # サーバーへの負荷を避けるために少し待機
    time.sleep(0.5)

# 最終的な出力行数を表示
print(f"\n出力された行数: {line_count - 1}")


1, 記入なし, 記入なし, 記入なし
2, 記入なし, 記入なし, 記入なし
3, 記入なし, 記入なし, 記入なし
4, 記入なし, 記入なし, 記入なし
5, 記入なし, 記入なし, 記入なし
6, 記入なし, 記入なし, 記入なし
7, 記入なし, 記入なし, 記入なし
8, 記入なし, 記入なし, 記入なし
9, 記入なし, 記入なし, 記入なし
10, 記入なし, 記入なし, 記入なし
11, 記入なし, 記入なし, 記入なし
12, 記入なし, 記入なし, 記入なし
13, 記入なし, 記入なし, 記入なし
14, 記入なし, 記入なし, 記入なし
15, 記入なし, 記入なし, 記入なし
16, 記入なし, 記入なし, 記入なし
17, 記入なし, 記入なし, 記入なし
18, 記入なし, 記入なし, 記入なし
19, 記入なし, 記入なし, 記入なし
20, 記入なし, 記入なし, 記入なし
21, 記入なし, 記入なし, 記入なし
22, 記入なし, 記入なし, 記入なし
23, 記入なし, 記入なし, 記入なし
24, 記入なし, 記入なし, 記入なし
25, 記入なし, 記入なし, 記入なし
26, 記入なし, 記入なし, 記入なし
27, 記入なし, 記入なし, 記入なし
28, 記入なし, 記入なし, 記入なし
29, 記入なし, 記入なし, 記入なし
30, 記入なし, 記入なし, 記入なし
31, 記入なし, 記入なし, 記入なし
32, 記入なし, 記入なし, 記入なし
33, 記入なし, 記入なし, 記入なし
34, 記入なし, 記入なし, 記入なし
35, 記入なし, 記入なし, 記入なし
36, 記入なし, 記入なし, 記入なし
37, 記入なし, 記入なし, 記入なし
38, 記入なし, 記入なし, 記入なし
39, 記入なし, 記入なし, 記入なし
40, 記入なし, 記入なし, 記入なし
41, 記入なし, 記入なし, 記入なし
42, 記入なし, 記入なし, 記入なし
43, 記入なし, 記入なし, 記入なし
44, 記入なし, 記入なし, 記入なし
45, 記入なし, 記入なし, 記入なし
46, 記入なし, 記入なし, 記入なし
47, 記入なし, 記入なし, 記入なし
48, 記入なし, 記入なし, 記入なし
4

KeyboardInterrupt: 